##### Copyright 2024 Kensho Technologies, LLC

# Anthropic Function Calling
**_Claude to retrieve data from the LLM-ready API using the kFinance python library!_**

What you'll need to run this notebook:

1.   kFinance credentials
2.   An Anthropic API key

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/kensho-technologies/llm-ready-api-examples/blob/main/function_calling/Anthropic_function_calling.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Install dependencies

In [ ]:
# install the latest version of kFinance package
%pip install https://kfinance.kensho.com/static/kensho_finance.tar.gz
# install the LLM Python package
%pip install anthropic

# Instantiate kFinance Client

In [ ]:
# import the kfinance client
from kensho_finance.kfinance import Client
# import standard libraries
import functools
import types
import json
import sys
# check if the current environment is a Google Colab
try:
  import google.colab
  IN_GOOGLE_COLAB = True
except:
  IN_GOOGLE_COLAB = False

# initialize the kfinance client with one of the following:
# 1. your kensho refresh token
# 2. your kensho client id and kensho private key
# 3. automated login (not accessible on Google Collab)
if IN_GOOGLE_COLAB:
    kensho_refresh_token = ""
    assert kensho_refresh_token != "", "kensho refresh token is empty! Make sure to enter your kensho refresh token above"
    kfinance_client = Client(refresh_token=kensho_refresh_token)

    # kensho_client_id = ""
    # kensho_private_key = ""
    # assert kensho_client_id != "", "kensho client id is empty! Make sure to enter your kensho client id above"
    # assert kensho_private_key != "", "kensho private key is empty! Make sure to enter your kensho private key above"
    # kfinance_client = Client(client_id=kensho_client_id, private_key=kensho_private_key)
else:
    kfinance_client = Client()

# the prompt given to all LLMs to instruct them how to use tools to call the kFinance API
SYSTEM_PROMPT = "You are an LLM trying to help financial analysts. Use the supplied tools to assist the user. Always use the `get_latest` function when asked about the last or most recent quarter or time period etc. Always use the `get_latest` function when a tool requires a time parameter and the time is unspecified in the question"
# the message shown to users to prompt them to input a message
USER_INPUT_PROMPT = "Enter your message and press the [return] key\n"

# Anthropic Function Calling

In [ ]:
# import Anthropic and necessary Antrhopic types
from anthropic import Anthropic
from anthropic.types import TextBlock, ToolUseBlock

# parse response message for text
def text_from_response_message(response_message):
  if response_message.content is None:
    return None
  if isinstance(response_message.content, list) and list(filter(lambda c: isinstance(c, TextBlock), response_message.content)) != []:
    return list(filter(lambda c: isinstance(c, TextBlock), response_message.content))[0].text
  elif not isinstance(response_message.content, list):
    return response_message.content.text
  return None

# parts response message for tool calls
def tool_calls_from_response_message(response_message):
  if not isinstance(response_message.content, list):
    return []
  return list(filter(lambda c: isinstance(c, ToolUseBlock,), response_message.content))

# the AntropicChat class is used to create a chat loop that automatically executes function calls
class AnthropicChat:
  def __init__(self, kfinance_client):
    # initialize Anthropic with your Anthropic API key
    anthropic_api_key = "" # replace with your own key
    assert anthropic_api_key != "", "Anthropic API key is empty! Make sure to enter your Anthropic API key above"
    # good client config for Anthropic. There are other ways to access Anthropic
    self.anthropic = Anthropic(
      api_key=anthropic_api_key
    )
    # initialize the kFinance client
    self.kfinance_client = kfinance_client
    # initialize tools and tool descriptions
    self.tools = kfinance_client.tools
    self.tool_descriptions = kfinance_client.anthropic_tool_descriptions
    # initialize the message history
    self.messages = []

  def print_response(self):
      """Print and return response"""
      # try to send the message history to Anthropic and get the response
      response = self.anthropic.messages.create(
        model="claude-3-opus-20240229", # you can use any Anthropic model that supports function calling
        system=SYSTEM_PROMPT,
        messages=self.messages,
        tools=self.tool_descriptions,
        max_tokens=2048,
      )
      response_message = response
      self.messages.append({"role": "assistant", "content": response_message.content})
      # if the response contains text , print out the text
      response_message_text = text_from_response_message(response_message)
      if response_message_text is not None:
        sys.stdout.write(response_message_text)

      return response_message

  def print_responses(self, user_input: str) -> list[str]:
      """Print responses and call tools"""
      # append the current user input as a message to the message history
      self.messages.append({"role": "user", "content": user_input})
      # get the Anthropic response to the message history
      response_message = self.print_response()
      # while the response has tool calls
      tool_calls = tool_calls_from_response_message(response_message)
      while tool_calls != []:
          # for each tool call, execute the function and arguments specified in the tool call
          # and append the output as a message to the message history
          for tool_call in tool_calls:
              function = tool_call.name
              arguments = tool_call.input
              # try to execute the function and arguments specified in the tool call
              try:
                  output = self.tools[function](**arguments)
                  # append the output as a message to the message history
                  self.messages.append(
                      {
                          "role": "user",
                          "content": [{
                              "type": "tool_result",
                              "content": str(output),
                              "tool_use_id": tool_call.id,
                        }]
                      }
                  )
              # if there's an exception thrown while executing the function,
              # append the exception as a message to the message history
              except Exception as e:
                  self.messages.append(
                      {
                          "role": "user",
                          "content": [{
                              "type": "tool_result",
                              "content": str(e),
                              "tool_use_id": tool_call.id,
                              "is_error": True,
                          }]
                      }
                  )
          # get a new response and parse tool calls
          response_message = self.print_response()
          tool_calls = tool_calls_from_response_message(response_message)
      return None

  def start_chatting(self) -> None:
      """Open chat shell"""
      # prompt for user input and get the Anthropic response in a loop
      while True:
          user_input = input(USER_INPUT_PROMPT)
          self.print_responses(user_input)
          sys.stdout.write("\n")

In [ ]:
# instantiate the AnthropicChat with the kfinance client
anthropic_chat = AnthropicChat(kfinance_client)
# start chatting with the Anthropic
anthropic_chat.start_chatting()